In [1]:
import os
import struct
import socket
import pickle
import time

import h5py
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

from torch.utils.data import Subset
from torch.autograd import Variable
import torch.nn.init as init
from model import ResNet18, ResNet50
from utils import get_metrics
import copy

/mnt/data/home/bishwa/mambaforge-pypy3/envs/multimodal/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_path = '../models/cifar10_data'

# Setup cpu
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = 'cuda:1'
torch.manual_seed(777)

# Setup client order
client_order = int(0)
print('Client starts from: ', client_order)

num_train_data = 50000

# Load data
from random import shuffle

transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

indices = list(range(50000))

part_tr = indices[num_train_data * client_order : num_train_data * (client_order + 1)]

train_set  = torchvision.datasets.CIFAR10(root=root_path, train=True, download=True, transform=transform)
train_set_sub = Subset(train_set, part_tr)
train_loader = torch.utils.data.DataLoader(train_set_sub, batch_size=64, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root=root_path, train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, num_workers=2)

x_train, y_train = next(iter(train_loader))
print(f'Train batch shape x: {x_train.size()} y: {y_train.size()}')
total_batch = len(train_loader)
print(f'Num Batch {total_batch}')


Client starts from:  0
Files already downloaded and verified
Files already downloaded and verified
Train batch shape x: torch.Size([64, 3, 32, 32]) y: torch.Size([64])
Num Batch 782


In [3]:
resnet_model = ResNet50(channel=3, num_classes=10).to(device)
epoch = 10
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_model.parameters(), lr = lr, momentum = 0.9)

# Start training
print("Start training @ ", time.asctime())

for epc in range(epoch):
    start_time = time.time()    
    for i, data in enumerate(tqdm(train_loader, ncols=100, desc='Centralized training', disable=True)):
        x, label = data
        x = x.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = resnet_model(x)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            # measure accuracy and record loss
            _, predicted = torch.max(output, 1)
            correct = (predicted == label).sum().item()
            accuracy = correct / len(label)
            print(f'Epoch: {epc+1}/{epoch}, Batch: {i+1}/{total_batch}, Train Loss: {round(loss.item(), 2)} Train Accuracy: {round(accuracy, 2)}')

        # if (i + 1) % 1000 == 0:
        #     test_loss, test_acc, test_auc, test_bal_acc = get_metrics(resnet_model, test_loader, device)
        #     print(f'                                           Test Loss: {round(test_loss, 2)} Test Accuracy: {round(test_acc, 2)} Test AUC: {round(test_auc, 2)} Test Balanced Accuracy: {round(test_bal_acc, 2)}')


    print(time.time() - start_time)

Start training @  Mon Oct 23 14:32:02 2023
Epoch: 1/10, Batch: 100/782, Train Loss: 2.42 Train Accuracy: 0.06
Epoch: 1/10, Batch: 200/782, Train Loss: 2.37 Train Accuracy: 0.06
Epoch: 1/10, Batch: 300/782, Train Loss: 2.38 Train Accuracy: 0.17
Epoch: 1/10, Batch: 400/782, Train Loss: 2.14 Train Accuracy: 0.28
Epoch: 1/10, Batch: 500/782, Train Loss: 2.27 Train Accuracy: 0.17
Epoch: 1/10, Batch: 600/782, Train Loss: 2.16 Train Accuracy: 0.17
Epoch: 1/10, Batch: 700/782, Train Loss: 2.05 Train Accuracy: 0.33
56.54062104225159
Epoch: 2/10, Batch: 100/782, Train Loss: 2.06 Train Accuracy: 0.25
Epoch: 2/10, Batch: 200/782, Train Loss: 1.84 Train Accuracy: 0.25
Epoch: 2/10, Batch: 300/782, Train Loss: 1.7 Train Accuracy: 0.44
Epoch: 2/10, Batch: 400/782, Train Loss: 2.03 Train Accuracy: 0.25
Epoch: 2/10, Batch: 500/782, Train Loss: 1.81 Train Accuracy: 0.33
Epoch: 2/10, Batch: 600/782, Train Loss: 2.11 Train Accuracy: 0.27
Epoch: 2/10, Batch: 700/782, Train Loss: 1.62 Train Accuracy: 0.47
56